#### Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.
​

In [2]:
import pandas as pd

In [3]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
ratings_sort = ratings.sort_index().sort_values('timestamp', kind='mergesort')
ratings_sort.head()

,userId,movieId,rating,timestamp
52635,383,21,3.0,789652009
52641,383,47,5.0,789652009
52684,383,1079,3.0,789652009
56906,409,16,4.0,828212412
56907,409,21,5.0,828212412


In [5]:
user = ratings.groupby(['userId']).count()
user.head()

,movieId,rating,timestamp
userId,,,
1,20,20,20
2,76,76,76
3,51,51,51
4,204,204,204
5,100,100,100


In [6]:
user_100 = user.loc[user.rating > 100]
user_100.head()

,movieId,rating,timestamp
userId,,,
4,204,204,204
8,116,116,116
15,1700,1700,1700
17,363,363,363
19,423,423,423


In [7]:
user_100 = user_100.reset_index()

In [8]:
user_100_id = user_100.loc[:, ['userId']]
user_100_id.head()

,userId
0,4
1,8
2,15
3,17
4,19


In [10]:
r = ratings_sort.loc[:, ['userId','timestamp']]
r.head()

,userId,timestamp
52635,383,789652009
52641,383,789652009
52684,383,789652009
56906,409,828212412
56907,409,828212412


In [11]:
r.drop_duplicates(subset = 'userId', keep = 'last', inplace = True)
r.rename(columns={'timestamp': 'last'}, inplace=True)
r.head()

,userId,last
73399,511,829352433
77388,536,829471925
54503,389,829495183
65331,464,829912558
56951,409,831090605


In [12]:
r2 = ratings_sort.loc[:, ['userId','timestamp']]
r2.head()

,userId,timestamp
52635,383,789652009
52641,383,789652009
52684,383,789652009
56906,409,828212412
56907,409,828212412


In [13]:
r2.drop_duplicates(subset = 'userId', keep = 'first', inplace = True)
r2.rename(columns={'timestamp': 'first'}, inplace=True)
r2.head()

,userId,first
52635,383,789652009
56906,409,828212412
31243,224,828213150
73371,511,829351964
77383,536,829471719


In [14]:
usertime = r.merge(r2, how='outer')
usertime

,userId,last,first
0,511,829352433,829351964
1,536,829471925,829471719
2,389,829495183,829491839
3,464,829912558,829911793
4,409,831090605,828212412
...,...,...,...
666,62,1475948734,1448760334
667,73,1476086345,1255500852
668,547,1476587644,974777109
669,624,1476616373,1019123866


In [15]:
usertime['time'] = usertime['last'] - usertime['first']
usertime

,userId,last,first,time
0,511,829352433,829351964,469
1,536,829471925,829471719,206
2,389,829495183,829491839,3344
3,464,829912558,829911793,765
4,409,831090605,828212412,2878193
...,...,...,...,...
666,62,1475948734,1448760334,27188400
667,73,1476086345,1255500852,220585493
668,547,1476587644,974777109,501810535
669,624,1476616373,1019123866,457492507


In [16]:
total = user_100_id.merge(usertime, how='left')
total

,userId,last,first,time
0,4,949982274,949778714,203560
1,8,1154474527,1154389340,85187
2,15,1469330735,997937239,471393496
3,17,1127476640,1127468587,8053
4,19,855195373,855190091,5282
...,...,...,...,...
253,656,986244044,986240991,3053
254,659,866207451,834598040,31609411
255,664,1441911722,1343731283,98180439
256,665,1046967769,992836298,54131471


#### Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента


Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [17]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [18]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [19]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [20]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [21]:
revenue = rzd.merge(auto, how='outer').merge(air, how='outer')
revenue

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [22]:
revenue_adress = revenue.merge(client_base, how='left')
revenue_adress

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


#### Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.
​

Зависит от того, какие данные есть помимо связки user_id и координаты. Если есть так же признак времени, когда эти координаты были получены, то можно связать по этому признаку с таблицей визитов и покупок, и получить сведения где конкретно была произведена покупка/куда произведен визит, если у нас вдруг не было этих сведений.
При внешнем объединении таблиц мы можем получить карту движения пользователя (если координаты запрашиваются достаточно часто), если есть постоянно повтряющиеся координаты, то можно предположить, что это дом или место работы пользователя, и далее анилизировать информацию в разрезе - пользователь целенаправлено отправился в наш магазин или это было спонтанное решение, когда он проходил мимо.